In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = d2l.synthetic_data(true_w,true_b,1000)

In [3]:
batch_size=10
dataset = data.TensorDataset(*(features,labels))
data_iter = data.DataLoader(dataset,batch_size,shuffle=True)

In [4]:
next(iter(data_iter))

[tensor([[ 0.9188,  0.6406],
         [-0.5707, -1.3351],
         [ 1.3931, -1.8306],
         [-1.7588,  0.3455],
         [ 0.5131, -0.9186],
         [-0.3804,  0.6678],
         [-0.7930,  1.7189],
         [-0.2104,  0.3236],
         [ 0.3419, -0.1770],
         [-1.0655,  0.3594]]),
 tensor([[ 3.8527],
         [ 7.6106],
         [13.2000],
         [-0.4824],
         [ 8.3691],
         [ 1.1700],
         [-3.2190],
         [ 2.6794],
         [ 5.4908],
         [ 0.8638]])]

In [5]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

In [6]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [7]:
loss = nn.MSELoss()

In [8]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

In [9]:
num_epochs= 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        #避免梯度叠加
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch+1},loss {l:f}')    

epoch 1,loss 0.000218
epoch 2,loss 0.000101
epoch 3,loss 0.000100


In [10]:
w = net[0].weight.data
print('w的估计误差',true_w-w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：',true_b-b)

w的估计误差 tensor([-2.0242e-04, -1.5736e-05])
b的估计误差： tensor([-0.0005])


## 练习 

In [11]:
# huber损失对应Pytorch的SmoothL1损失
loss = nn.SmoothL1Loss(beta=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        #开始计算梯度
        trainer.zero_grad()
        l.backward()
        trainer.step()  #对所有参数进行更新
    print("epoch: {}, loss:{}".format(epoch + 1, l))

epoch: 1, loss:4.98314511787612e-05
epoch: 2, loss:3.757154263439588e-05
epoch: 3, loss:5.637608774122782e-05


In [12]:
net[0].weight.grad,net[0].bias.grad

(tensor([[-0.0012, -0.0030]]), tensor([-0.0116]))

In [13]:
net[0].weight

Parameter containing:
tensor([[ 1.9987, -3.3995]], requires_grad=True)